In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from tqdm.notebook import trange, tqdm

from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as Func
from torch.utils.data import DataLoader, TensorDataset
# Only this extra line of code is required to use oneDNN Graph
torch.jit.enable_onednn_fusion(True)
torch.autograd.detect_anomaly = False
torch.autograd.profiler.emit_nvtx = False 
torch.autograd.profiler.profile = False
torch.autograd.gradcheck = False
torch.autograd.gradgradcheck = False

data_DIR = "../data/rapm/"
misc_DIR = "../data/misc/"
model_path = "../data/models/"
pbp_DIR = "../data/pbpdata/"
fig_DIR = "../figs/analysis/"

# %matplotlib widget

## Data Pre Processing

In [ ]:
# loads possessions with odds
dfw = pd.read_parquet(data_DIR + "NBA_rapm_possessions_odds_2017_2024.parquet")
len(dfw)


In [ ]:
# random seed
rr = 11

In [ ]:
X = dfw[['margin', 'spread', 'secs']].values
y = dfw['win'].values

# sample the data
test_gid = dfw['gid'].sample(frac=0.8, random_state=rr).to_list()
dfw11 = dfw[dfw['gid'].isin(test_gid)]
# dfw12 = dfw11.query("secs <=120")
# for i in range(4):
#     dfw11  = pd.concat([dfw11,dfw12])
dfw1 = dfw11
dfw2 = dfw[dfw['gid'].isin(test_gid)]

# scale the data
scaler = MinMaxScaler()
smodel = scaler.fit(X)
Xs = smodel.transform(X)
X_train =  smodel.transform(dfw1[['margin', 'spread', 'secs']].values)
y_train = dfw1['win'].values
X_test =  smodel.transform(dfw2[['margin', 'spread', 'secs']].values)
y_test = dfw2['win'].values

# convert to tensors
inputs = torch.FloatTensor(Xs)
labels = torch.FloatTensor(y).unsqueeze(1)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train).unsqueeze(1)
y_test = torch.FloatTensor(y_test).unsqueeze(1)

## No Batches

In [ ]:
def try_weights(h1=12,h2=12,num_epochs=100,des="base"):
    torch.manual_seed(rr)
    # Initialize the model
    model = nn.Sequential(
        nn.Linear(3,h1),
        nn.ReLU(),
        nn.Linear(h1,h2),
        nn.ReLU(),
        nn.Linear(h2,1),
        nn.Sigmoid()
    )
    criterion = nn.BCELoss()
    # optimizer = torch.optim.RMSprop(model.parameters(),lr=1e-3)
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    losses = []
    for epoch in trange(num_epochs,desc="epochs",leave=False):
        model.train()
        # optimizer.zero_grad()
        for param in model.parameters():
            param.grad = None
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    pred = model.forward(inputs)
    x_out = dfw["secs"].values
    y_out = pred.detach().numpy()
    fig,ax=plt.subplots(1,1,figsize=(5,4))
    ax.plot(range(num_epochs),losses)
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
    fig.tight_layout()
    plt.savefig(f"./testing/weights_{h1:02d}_{h2:02d}_{num_epochs:03d}_{des}_losses.png",dpi=200)
    fig,ax=plt.subplots(1,1,figsize=(5,4))
    ax.plot(x_out,y_out,"x")
    ax.set_xlim((3000,0))
    ax.set_xlabel("Time Remaining in Game [s]")
    ax.set_ylabel("Win Probability")
    ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
    fig.tight_layout()
    plt.savefig(f"./testing/weights_{h1:02d}_{h2:02d}_{num_epochs:03d}_{des}_res.png",dpi=200)
    torch.save(model, f"./testing/model_{h1:02d}_{h2:02d}_{num_epochs:03d}_{des}.pt")
    return model, losses

### Evaluation

In [ ]:
# model, losses = try_weights(h1=12,h2=12,num_epochs=50,des="opt")

In [ ]:
for h1 in trange(12,25,4, desc="h1"):
    for h2 in trange(h1,int(h1/2)-1,-4, desc="h2", leave=False): 
        try_weights(h1=h1,h2=h2,num_epochs=400,des="opt")

## Batches

In [ ]:
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler

In [ ]:
torch.multiprocessing.get_all_sharing_strategies()

In [ ]:
# Create DataLoader for efficient batching
bs=8
dataset = TensorDataset(inputs, labels)
batch_size = int(len(dataset)/bs)+1
world_size = 4
rank = 0
num_epochs = 50
os.environ['MASTER_ADDR'] = '192.168.1.3'
os.environ['MASTER_PORT'] = '8888'

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print("Number of GPUs:", torch.cuda.device_count())
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

In [ ]:
def try_weights_dist(h1=12,h2=12,batch_size=batch_size,rank=rank, world_size=world_size):
    torch.manual_seed(rr)
    # Initialize the model
    model = nn.Sequential(
        nn.Linear(3,h1),
        nn.ReLU(),
        nn.Linear(h1,h2),
        nn.ReLU(),
        nn.Linear(h2,1),
        nn.Sigmoid()
    )
    # Distributed
    dist.init_process_group("gloo", rank=rank, world_size=world_size)
    # construct DDP model
    model = DDP(model, device_ids=[rank])
    
    # loss function
    criterion = nn.BCELoss()
    # optimizer = torch.optim.RMSprop(model.parameters(),lr=1e-3)
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    batch_size = batch_size
    sampler = DistributedSampler(dataset,num_replicas=world_size,rank=rank)
    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
    losses = []
    for epoch in range(num_epochs,desc="epochs"):
        model.train()
        for i, batch in enumerate(dataloader):
            inputs_batch,labels_batch = batch
            for param in model.parameters():
                param.grad = None
            outputs = model(inputs_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
                clear_output(wait=True)
                print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')

    return model, losses

In [ ]:
# try_weights_dist(h1=12,h2=12,batch_size=batch_size,rank=rank, world_size=world_size)
mp.spawn(try_weights_dist,nprocs=world_size)

In [ ]:


dataloader = DataLoader(dataset, batch_size=batch_size)#,num_workers=1)
num_epochs=100

In [ ]:
for h1 in trange(12,13,4, desc="h1"):
    for h2 in trange(h1,int(h1/2)-1,-4, desc="h2"): 
        try_weights(h1=h1,h2=h2,num_epochs=50,bs=4,des="opt")

## Batches No Parallel

In [ ]:
def try_weights(h1=12,h2=12,num_epochs=100,bs=4,des="base",rank=rank, world_size=world_size):
    torch.manual_seed(rr)
    # Initialize the model
    model = nn.Sequential(
        nn.Linear(3,h1),
        nn.ReLU(),
        nn.Linear(h1,h2),
        nn.ReLU(),
        nn.Linear(h2,1),
        nn.Sigmoid()
    )
    # Distributed
    dist.init_process_group("gloo", rank=rank, world_size=world_size)
    ddp_model = DDP(model, device_ids=[rank])
    # Create DataLoader for efficient batching
    dataset = TensorDataset(inputs, labels)
    batch_size = int(len(dataset)/bs)+1
    dataloader = DataLoader(dataset, batch_size=batch_size)#,num_workers=1)
    # loss function
    criterion = nn.BCELoss()
    # optimizer = torch.optim.RMSprop(model.parameters(),lr=1e-3)
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    losses = []
    for epoch in trange(num_epochs,desc="epochs"):
        model.train()
        for i in trange(len(dataloader),desc="batches", leave=False):
            inputs_batch,labels_batch = next(iter(dataloader))
            for param in model.parameters():
                param.grad = None
            outputs = model(inputs_batch)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            if i % 10 == 0:
                clear_output(wait=True)
                print(f'Epoch {epoch}/{num_epochs}, Loss: {loss.item()}')
    
    pred = model.forward(inputs)
    x_out = dfw["secs"].values
    y_out = pred.detach().numpy()
    fig,ax=plt.subplots(1,1,figsize=(5,4))
    ax.plot(range(num_epochs*bs),losses)
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
    fig.tight_layout()
    plt.savefig(f"./testing/weights_{h1:02d}_{h2:02d}_{num_epochs:03d}_{des}_losses.png",dpi=200)
    fig,ax=plt.subplots(1,1,figsize=(5,4))
    ax.plot(x_out,y_out,"x")
    ax.set_xlim((3000,0))
    ax.set_xlabel("Time Remaining in Game [s]")
    ax.set_ylabel("Win Probability")
    ax.set_title(f"Layer 1: {h1:02d} Layer2: {h2:02d} Epochs: {num_epochs:03d}")
    fig.tight_layout()
    plt.savefig(f"./testing/weights_{h1:02d}_{h2:02d}_{num_epochs:03d}_batch{batch_size:02d}_{des}_res.png",dpi=200)
    torch.save(model, f"./testing/model_{h1:02d}_{h2:02d}_{num_epochs:03d}_batch{batch_size:02d}_{des}.pt")
    return model, losses


In [ ]:
for h1 in trange(12,13,4, desc="h1"):
    for h2 in trange(h1,int(h1/2)-1,-4, desc="h2"): 
        try_weights(h1=h1,h2=h2,num_epochs=50,bs=4,des="opt")

In [ ]:
dfgfdg

In [ ]:
pred = model.forward(inputs)
x_out = dfw["secs"].values
y_out = pred.detach().numpy()

In [ ]:
dfw["win_prob"] = y_out
games  = dfw["gid"].unique()

In [ ]:
team_dict, team_list  = get_teams()

In [ ]:
line =  -15
dfw1 = dfw.query(f"spread == {line}")
p = (
    ggplot(dfw1)
    + aes(x="secs",y="win_prob",group="gid")
    + geom_smooth(se=False,size=0.2)
    + scale_x_reverse()
    + scale_y_continuous(labels=percent_format())
    + theme_xkcd(base_size=16,stroke_size=0.1)
    + labs(title = f"{team_dict[dfw1["tida"].iloc[0]]} vs {team_dict[dfw1["tidh"].iloc[0]]}")
)
p

In [ ]:
sadasdsa

In [ ]:
with torch.no_grad():
    y_eval = model.forward(X_test)
    loss = criterion(y_eval,y_test)
print(loss)
torch.save(model.state_dict(), model_path +"win_prob_dict_1")
torch.save(model, model_path +"win_prob_1.pt")
# Model class must be defined somewhere
# model1 = torch.load( model_path +"win_prob_1.pt", weights_only=False)
# model1.eval()

In [ ]:
league = "nba"
margin = -5 # from the perspective of the team with the ball
seconds_remaining = 60 # This is time remaining at the start of the possession
pre_game_win_prob = 0.55 # 55% to win pregame
end_of_possession_seconds_remaining = 45 # seconds remaining at end of possession

In [ ]:
url = f"https://api.pbpstats.com/get-leverage/{league}/{margin}/{pre_game_win_prob}/{seconds_remaining}/{end_of_possession_seconds_remaining}"

response = requests.get(url)
response_json = response.json()
response_json

In [ ]:
# # Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
# model1 = PyTorchModel()
# model1.load_state_dict(torch.load(model_path +"win_prob_dict_1", weights_only=True))
# model1.eval()

In [ ]:
for epoch in trange(num_epochs,desc="epochs", leave=False):
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        losses.append(loss.detach().numpy())
        # if epoch % 10 == 0:
        #     print(f'Epoch {epoch}/{num_epochs}, Loss: {loss}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # clear_output(wait=True)